In [15]:
import os
import time
import typing
from pathlib import Path
from textwrap import dedent

import numpy as np
from pydrake.all import (
    Concatenate,
    Context,
    DiagramBuilder,
    InverseKinematics,
    MultibodyPlant,
    PiecewisePolynomial,
    PointCloud,
    Rgba,
    RigidTransform,
    RotationMatrix,
    Simulator,
    Solve,
    StartMeshcat,
    TrajectorySource,
    ConstantVectorSource
)

from manipulation import running_as_notebook
# from manipulation.letter_generation import create_sdf_asset_from_letter
from manipulation.meshcat_utils import AddMeshcatTriad
from manipulation.station import LoadScenario, MakeHardwareStation, AddPointClouds
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from manipulation.utils import RenderDiagram

from scene_utils import *

meshcat = StartMeshcat()


INFO:drake:Meshcat listening for connections at http://localhost:7002


In [201]:
from pydrake.all import LeafSystem, ExternallyAppliedSpatialForce, SpatialForce, AbstractValue

class Shaker(LeafSystem):
    def __init__(self, plant):
        super().__init__()

        self._Kp = 200
        self._level_length = 5

        self.body = plant.GetRigidBodyByName("platform_link")
        # self.plant = plant

        # Input port to get all body poses
        self.DeclareAbstractInputPort(
            "poses",
            plant.get_body_poses_output_port().Allocate()
        )

        self.DeclareAbstractOutputPort(
            "force", 
            alloc = lambda: AbstractValue.Make([ExternallyAppliedSpatialForce()]),
            calc = self.CalcOutput
        )
    
    def CalcOutput(self, context : Context, output):
        t = context.get_time()
        level = t // self._level_length + 1

        X_WB_all = self.get_input_port(0).Eval(context)
        X_WB = X_WB_all[self.body.index()]
        x = X_WB.translation()[0]

        goal_x = np.sin(8 * np.pi * t) * 0.015 * level
        F_x = self._Kp * (goal_x - x)

        force = ExternallyAppliedSpatialForce()
        force.body_index = self.body.index()
        force.F_Bq_W = SpatialForce(tau= np.array([0, 0, 0]), f=np.array([F_x, 0, 0]))
        force.p_BoBq_B = np.array([0, 0, 0])

        output.set_value([force])

In [230]:
directives = """
directives:
    - add_model:
        name: floor
        file: package://stackbot/smooth_floor.sdf
    - add_weld:
        parent: world
        child: floor::floor_link
        X_PC:
            translation: [0, 0, -0.05]
    - add_model:
        name: platform
        file: package://stackbot/round_platform.sdf
"""

stack = "../assets/contexts/stack_rrt_678.json"

block_directives = create_block_directives_from_file(stack)
scenario_yaml = directives + block_directives

In [231]:
scenario = LoadScenario(data=scenario_yaml)
station = MakeHardwareStation(
    scenario=scenario,
    meshcat=meshcat,
    package_xmls=["../assets/models/package.xml"],
)
plant : MultibodyPlant = station.GetSubsystemByName("plant")

builder = DiagramBuilder()
builder.AddSystem(station)

shaker = builder.AddSystem(Shaker(plant))

builder.Connect(shaker.get_output_port(0), station.GetInputPort("applied_spatial_force"))
builder.Connect(station.GetOutputPort("body_poses"), shaker.get_input_port())

diagram = builder.Build()

In [232]:
import json

with open(stack, "r") as f:
    block_positions = json.load(f)

max_z = -np.inf
top_block = None
for block in block_positions:
    block_z = block_positions[block]["translation"][2]
    if block_z > max_z:
        max_z = block_z
        top_block = block

if "link" not in top_block:
    top_block = f"{top_block}_link"
print(top_block, max_z)

def z_height(plant : MultibodyPlant, plant_context, body_name):
    return plant.GetFreeBodyPose(plant_context, plant.GetRigidBodyByName(body_name)).translation()[2]


block5_link 0.17630399747833708


In [ ]:
diagram_context = diagram.CreateDefaultContext()
plant_context = diagram.GetMutableSubsystemContext(plant, diagram_context)
simulator = Simulator(diagram, diagram_context)

# run simulation!
meshcat.StartRecording()
if running_as_notebook:
    simulator.set_target_realtime_rate(4.0)

for level in range(1, 13):
    simulator.AdvanceTo(5 * level)
    stack_top = z_height(plant, plant_context, top_block)
    if stack_top < max_z - 0.1:
        print("collapsed at level", level)
        break
    
meshcat.StopRecording()
meshcat.PublishRecording()

collapsed at level 4


In [234]:
# RenderDiagram(diagram)